In [22]:
import pandas as pd
import numpy as np

In [23]:
file=pd.read_csv("ConsumptionRateMinMax.csv",low_memory=False)

In [24]:
file.columns

Index(['Unnamed: 0', 'Site', 'Item', 'Site_Item', 'Description',
       'SFI Group ID', 'Default Bin', 'Manufacturer', 'Manufacturer Part #',
       'Commodity Group', 'Minimum Level', 'Maximum Level', 'Condition Code',
       'Issue Unit', 'Current Balance', 'Qty Avbl', 'Qty Res'vd',
       'Average Cost', 'Ext'd Cost', 'Last Issue Date', 'Last Receipt Date',
       'Site_item_key', 'Key', 'Consumption', 'Consumption Rate',
       'New Minimum', 'New Maximum'],
      dtype='object')

In [25]:
commodity_type=list(file['Commodity Group'].unique())

In [26]:
commodity_type

['CON', 'ESP', 'LDE', 'AST', nan]

In [27]:
def excessCostForCommodityType(filename):
    import pandas as pd
    import numpy as np
    import re
    columns=['Site','Commodity Type', 'Excess Cost','Total Excess']
   
    data_values = []
    
    inventoryDataFrame=pd.DataFrame(filename)
    
    # removing the commas
    
    inventoryDataFrame['Current Balance'] = inventoryDataFrame['Current Balance'].replace(to_replace =",", value = '', regex = True) 

    inventoryDataFrame['New Maximum'] = inventoryDataFrame['New Maximum'].replace(to_replace =",", value = '', regex = True) 

    inventoryDataFrame['New Minimum'] = inventoryDataFrame['New Minimum'].replace(to_replace =",", value = '', regex = True) 

    inventoryDataFrame['Average Cost'] = inventoryDataFrame['Average Cost'].replace(to_replace =",", value = '', regex = True) 

    # converting to float for calculations
    
    inventoryDataFrame['Average Cost'] = inventoryDataFrame['Average Cost'].astype(float)

    inventoryDataFrame['New Minimum'] = inventoryDataFrame['New Minimum'].astype(float)

    inventoryDataFrame['New Maximum'] = inventoryDataFrame['New Maximum'].astype(float)

    inventoryDataFrame['Current Balance'] = inventoryDataFrame['Current Balance'].astype(float)
    site=inventoryDataFrame['Site'].unique()
    commodity_type=list(inventoryDataFrame['Commodity Group'].unique())
    rigs_data=[]*len(site)
    # excess inventory
    for Site in site:
        inventoryDataFrame['Excess Inventory']=inventoryDataFrame['Current Balance']-inventoryDataFrame['New Maximum']

        RigsExcessInventoryCount=inventoryDataFrame.loc[inventoryDataFrame['Excess Inventory'] >0, 'Excess Inventory'].sum()

        #excess cost
        inventoryDataFrame['Excess Cost']= inventoryDataFrame['Average Cost']* inventoryDataFrame['Excess Inventory']

        RigsExcessCostSumMaxND=(inventoryDataFrame.loc[(inventoryDataFrame['Excess Cost'] >0) & (inventoryDataFrame['New Maximum'] !=0) &(inventoryDataFrame['Site']==Site),'Excess Cost'].sum())


        RigCommodityExcessCostSumMaxND=[]
        for commodity in commodity_type:
           # print(commodity)
            Excess_cost_for_commodity=(inventoryDataFrame.loc[(inventoryDataFrame['Excess Cost'] >0) & (inventoryDataFrame['New Maximum'] !=0)&(inventoryDataFrame['Commodity Group']==commodity)&(inventoryDataFrame['Site']==Site),'Excess Cost'].sum())
            #print(Excess_cost_for_commodity)
            
            data_values.append([Site,commodity,Excess_cost_for_commodity,RigsExcessCostSumMaxND])
            #print(data_values)
            
        
        rig_data_frame = pd.DataFrame(data_values, columns=columns)
        
    
    return (rig_data_frame)

In [28]:
file.columns

Index(['Unnamed: 0', 'Site', 'Item', 'Site_Item', 'Description',
       'SFI Group ID', 'Default Bin', 'Manufacturer', 'Manufacturer Part #',
       'Commodity Group', 'Minimum Level', 'Maximum Level', 'Condition Code',
       'Issue Unit', 'Current Balance', 'Qty Avbl', 'Qty Res'vd',
       'Average Cost', 'Ext'd Cost', 'Last Issue Date', 'Last Receipt Date',
       'Site_item_key', 'Key', 'Consumption', 'Consumption Rate',
       'New Minimum', 'New Maximum'],
      dtype='object')

In [29]:
rigs_data_consuption_min_max=excessCostForCommodityType(file)

In [30]:
rigs_data_consuption_min_max.head()

,Site,Commodity Type,Excess Cost,Total Excess
0,2020.0,CON,402957.70,1707951.94
1,2020.0,ESP,1299314.66,1707951.94
2,2020.0,LDE,5679.58,1707951.94
3,2020.0,AST,0.00,1707951.94
4,2020.0,NaN,0.00,1707951.94


In [31]:
rigs_data_consuption_min_max.isnull().sum()

Site               5
Commodity Type    23
Excess Cost        0
Total Excess       0
dtype: int64

In [34]:
rigs_data_consuption_min_max.dropna(inplace=True)

In [38]:
rigs_data_consuption_min_max.to_csv("rigs_data_consuption_min_max.csv",index=False)

In [39]:
rigs_data1=pd.read_csv("rigs_data_consuption_min_max.csv")
    

In [47]:
rigs_data1.shape

(88, 4)

In [48]:
rigs_name=pd.read_csv("SeadrillRigs.csv")

In [49]:
rigs_name.head()

,ID,Rig Name,Operating,Country,Region,Model
0,2313,AOD 1,Operating,KSA,KSA,KFELS
1,2314,AOD 2,Operating,KSA,KSA,KFELS
2,2315,AOD 3,Operating,KSA,KSA,KFELS
3,2306,T15,Operating,Thailand,FEA,Tender
4,2307,T16,Not Operating,Thailand,FEA,Tender


In [45]:
Rig_Data_Classfication_consuption_min_max=pd.merge(rigs_data1, rigs_name, left_on='Site', right_on='ID', how='left').drop('ID', axis=1)

In [50]:
Rig_Data_Classfication_consuption_min_max.shape

(88, 9)

In [51]:
Rig_Data_Classfication_consuption_min_max.dropna()

,Site,Commodity Type,Excess Cost,Total Excess,Rig Name,Operating,Country,Region,Model
0,2020.0,CON,4.029577e+05,1.707952e+06,West Epsilon,Not Operating,Europe,NADL,Gusto CJ 70
1,2020.0,ESP,1.299315e+06,1.707952e+06,West Epsilon,Not Operating,Europe,NADL,Gusto CJ 70
2,2020.0,LDE,5.679580e+03,1.707952e+06,West Epsilon,Not Operating,Europe,NADL,Gusto CJ 70
3,2020.0,AST,0.000000e+00,1.707952e+06,West Epsilon,Not Operating,Europe,NADL,Gusto CJ 70
4,2090.0,CON,1.117659e+06,3.741356e+06,West Elara,Operating,Europe,NADL,Gusto CJ 70
5,2090.0,ESP,2.371231e+06,3.741356e+06,West Elara,Operating,Europe,NADL,Gusto CJ 70
6,2090.0,LDE,2.524658e+05,3.741356e+06,West Elara,Operating,Europe,NADL,Gusto CJ 70
7,2090.0,AST,0.000000e+00,3.741356e+06,West Elara,Operating,Europe,NADL,Gusto CJ 70
8,2298.0,CON,7.427571e+05,2.860465e+06,West Tucana,Operating,Qatar,KSA,JU 2000 E
9,2298.0,ESP,2.108988e+06,2.860465e+06,West Tucana,Operating,Qatar,KSA,JU 2000 E


In [52]:
Rig_Data_Classfication_consuption_min_max.shape

(88, 9)

In [53]:
Rig_Data_Classfication_consuption_min_max.to_csv("Rig_Data_Classfication_consuption_min_max.csv",index=False)